<a href="https://colab.research.google.com/github/exeea/stable-diffusion-grpcserver-colab/blob/main/GRPCServer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## System Information

In [ ]:
! nvidia-smi
! nvcc -V
! free -h
!echo "-----"
!df -h

## Installation

In [ ]:
# @markdown # Install the Environment
import os
import sys
!apt update
!apt upgrade -y
!python --version

if not os.path.exists("/usr/local/bin/conda"):
  print("Installing conda...")
  !wget -O mini.sh https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
  !chmod +x mini.sh
  !bash ./mini.sh -b -f -p /usr/local
  !rm mini.sh
  !conda install -q -y jupyter
  !conda install -q -y google-colab -c conda-forge
  !python -m ipykernel install --name "py39" --user
  !apt install python-pip -y
  !python --version
  print("Conda is ready.")
else:
  print("Conda already installed.")

!conda update -n base -c defaults conda -y
print("done.")

In [ ]:
# @markdown # Install the GRPCServer
# @markdown Note: xformers are not yet working.
%cd /content
!git clone --depth=1 https://github.com/hafriedlander/stable-diffusion-grpcserver.git
%cd stable-diffusion-grpcserver
os.environ['PIP_EXTRA_INDEX_URL'] = "https://download.pytorch.org/whl/cu116"
os.environ['FLIT_ROOT_INSTALL'] = "1"

install_script = """#!/bin/bash
  eval "$(conda shell.bash hook)"
  cd /content/stable-diffusion-grpcserver
  conda env create -f environment.yaml
  conda activate sd-grpc-server
  flit install --pth-file
  conda install xformers -c xformers/label/dev
  python3 -m pip install --upgrade tensorrt"""
! {install_script}

print("done.")

## Start service

In [ ]:
# @markdown # Run script
# @markdown keep in mind that this script is set to run for ever.

# @markdown The HuggingFace API Key you have to get it from https://huggingface.co/settings/tokens, if you don't have one: create it with "write" permissions.
HUGGINGFACE_API_KEY = "hf_your_huggingface_api_token" # @param {type: "string"}
# @markdown The Access Token is your secret API key that you will use to connect. This is up to you.
ACCESS_TOKEN = "your_secret" # @param {type: "string"}
NSFW_BEHAVIOUR = "ignore" #@param ["ignore","flag","block"]


# @markdown when the server is started, before the models are loaded, you will get a message with:
# @markdown > GRPC listening on port 50051, HTTP listening on port 5000. Start your engines....\
# @markdown > Loading engines...
# @markdown > - Engine stable-diffusion-v1-5-standard...
# @markdown > - Model laion-clip-b...\
# @markdown > Localtunnel started. Use these settings to connect:
# @markdown >>  Server 'this-is-an-example-54-121-190-94.loca.lt'\
# @markdown >>  Port '443'\
# @markdown >>  Key 'your_secret'
# @markdown >>  
# @markdown >
# @markdown ## Use those informations to connect: Server, Port and Key
# @markdown \
# @markdown ## is possible that you will have to navigate, using your browser, to https://that_server_sddress_you_get and press the CONTINUE before be able to use the endpoint correctly

import os
%cd /content/stable-diffusion-grpcserver
! git pull
!echo "Machine IP: "
!curl ipecho.net/plain
!echo ""
os.environ['PIP_EXTRA_INDEX_URL'] = "https://download.pytorch.org/whl/cu116"
os.environ['SD_LISTEN_TO_ALL'] = "1"
os.environ['SD_LOCALTUNNEL'] = "1"
os.environ['HF_API_TOKEN'] = HUGGINGFACE_API_KEY
os.environ['SD_ACCESS_TOKEN'] = ACCESS_TOKEN
os.environ['SD_NSFW_BEHAVIOUR'] = NSFW_BEHAVIOUR
run_script = """#!/bin/sh
  eval "$(conda shell.bash hook)"
  conda activate sd-grpc-server
  python -V
  python server.py"""
! {run_script}

Some notes:
### idea2.art: 
- you have to fill up server, port and key as written (no http or https in front of the dns)

### flying dog PS/Krita plugin: 
- you have to put https:// in front of the dns, for example https://flying-server-324.34lr4.loca.lt (the port is not needed, https is 443)